In [1]:
from music21 import converter, instrument, note, chord, 
import numpy as np
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pickle
from collections import defaultdict

In [7]:
def read_midi(file_path): 
    print(file_path)
    notes = [] 
    notes_to_parse = None 
    instruments = []
    midi= converter.parse(file_path)

    s2 = instrument.partitionByInstrument(midi)
    for part in s2.parts:
        str_part = str(part)
        if str_part == '<music21.stream.Part>': 
            continue 
        else:

            cutoff_len = len('<music21.stream.Part ') 
            instru = str_part[cutoff_len:]
            instru = instru[:instru.find('>')]
            notes_to_parse = part.recurse() 
            for element in notes_to_parse: 
                if isinstance(element, note.Note): 
                    notes.append(str(element.pitch))
                    instruments.append(instru)
                elif isinstance(element, chord.Chord): 
                    notes.append('.'.join(str(n) for n in element.normalOrder))      
                    instruments.append(instru)
    return np.array(list(zip(instruments, notes)))

            
                    


def unique(np_array, array_type, unique_set): 
    unique_set_reverse = {n:idx for (idx, n) in unique_set.items()}
    new_array =[] 
    if array_type == 'x':
        for group in np_array: 
            temp = [] 
            for n in group: 
                n = ' '.join(n)
                temp.append(unique_set_reverse[n])
            new_array.append(temp)
    else: 
        for n in np_array: 
            n = ' '.join(n)
            new_array.append(unique_set_reverse[n])
    new_array = np.array(new_array)
    return new_array


def get_tts(base_dir, pick_name = None, timestep = 10): 
    song_files = [f'{base_dir}/{i}' for i in os.listdir(base_dir)]
    songs_notes = [read_midi(i) for i in tqdm(song_files, desc = 'Reading Midi Files')]
    all_notes = [' '.join(i) for note_ in songs_notes for i in note_]
    note_counter = defaultdict(int)
    for i in all_notes: 
        note_counter[i]+= 1
        
    all_notes_dict = {idx: n for idx, n in enumerate(set(all_notes))}
    
    x = [] 
    y = [] 

    pbar = tqdm(songs_notes, desc = 'Creating Timeseries')
    for idx_song, song in enumerate(pbar):
        for idx in range(timestep,len(song)-(timestep), 1): 
            x.append(song[idx:idx+timestep])
            y.append(song[idx + (timestep)])

    x = np.array(x)
    y = np.array(y)  
    
    new_x = unique(x, unique_set = all_notes_dict, array_type = 'x')
    new_y = unique(y, unique_set = all_notes_dict, array_type ='y')
    
    
    x_train, x_test, y_train, y_test = train_test_split(new_x, new_y, train_size = .90, random_state = 10)
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1],1)
    
    if pick_name:
        pick_tup= (x_train, x_test, y_train, y_test, all_notes_dict)
        pickle.dump(pick_tup, open(f'../Pickles/{pick_name}.p', 'wb'))
                    
    return x_train, x_test, y_train, y_test, all_notes_dict

x_train, x_test, y_train, y_test, notes_dict = get_tts('../Techno', pick_name = 'Techno')

Reading Midi Files:   3%|███▎                                                                                                                              | 1/39 [00:00<00:05,  6.94it/s]

../Techno/abridge_d.mid
../Techno/ACKER-GS.mid


Reading Midi Files:   5%|██████▋                                                                                                                           | 2/39 [00:01<00:15,  2.43it/s]

../Techno/aintnostoppingusnow.mid


Reading Midi Files:   8%|██████████                                                                                                                        | 3/39 [00:02<00:27,  1.32it/s]

...

Reading Midi Files:  95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍      | 37/39 [00:44<00:02,  1.03s/it]

../Techno/d-dontcryformeargentina.mid


Reading Midi Files:  97%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋   | 38/39 [00:46<00:01,  1.42s/it]

../Techno/d-dontgoaway.mid


Creating Timeseries: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 906.78it/s]


In [8]:
x_train

array([[[ 971],
        [ 380],
        [ 789],
        ...,
        [ 717],
        [ 294],
        [ 971]],

       [[ 299],
        [ 412],
        [ 717],
        ...,
        [ 771],
        [ 508],
        [ 717]],

       [[ 626],
        [ 590],
        [ 256],
        ...,
        [  90],
        [ 706],
        [ 332]],

       ...,

       [[  76],
        [ 766],
        [  50],
        ...,
        [  76],
        [ 558],
        [  76]],

       [[1086],
        [ 836],
        [ 233],
        ...,
        [ 836],
        [ 592],
        [1130]],

       [[  85],
        [  85],
        [  85],
        ...,
        [  85],
        [  85],
        [  85]]])

In [ ]:
def read_midi(file_path): 
    notes = [] 
    notes_to_parse = None 


    midi= converter.parse(file_path)
    
    s2 = instrument.partitionByInstrument(midi)
    for part in s2.parts:
    
        #select elements of only piano
        if 'Piano' in str(part): 
        
            notes_to_parse = part.recurse() 
      
            #finding whether a particular element is note or a chord
            for element in notes_to_parse:
                
                #note
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                
                #chord
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))

    return np.array(notes)

def unique(np_array, array_type, unique_set): 
    unique_set_reverse = {n:idx for (idx, n) in unique_set.items()}
    new_array =[] 
    if array_type == 'x':
        for group in np_array: 
            temp = [] 
            for n in group: 
                temp.append(unique_set_reverse[n])
            new_array.append(temp)
    else: 
        for n in np_array: 
            new_array.append(unique_set_reverse[n])
    new_array = np.array(new_array)
    return new_array

def get_tts(base_dir, pick_name = None, timestep = 10): 
    song_files = [f'{base_dir}/{i}' for i in os.listdir(base_dir)]
    songs_notes = [read_midi(i) for i in tqdm(song_files, desc = 'Reading Midi Files')]
    all_notes = [i for note_ in songs_notes for i in note_]
    note_counter = defaultdict(int)
    return songs_notes
    for i in all_notes: 
        note_counter[i]+= 1
    
    int_to_note = dict((idx, n) for idx, n in enumerate(all_notes))
    all_notes_dict = {idx: n for idx, n in enumerate(set(all_notes))}
    x = [] 
    y = [] 

    pbar = tqdm(songs_notes, desc = 'Creating Timeseries')
    for idx_song, song in enumerate(pbar):
        for idx in range(timestep,len(song)-(timestep), 1): 
            x.append(song[idx:idx+timestep])
            y.append(song[idx + (timestep)])

    x = np.array(x)
    y = np.array(y)  
    
    new_x = unique(x, unique_set = all_notes_dict, array_type = 'x')
    new_y = unique(y, unique_set = all_notes_dict, array_type ='y')
    
    
    x_train, x_test, y_train, y_test = train_test_split(new_x, new_y, train_size = .90, random_state = 10)
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1],1)
    
    if pick_name:
        pick_tup= (x_train, x_test, y_train, y_test, all_notes_dict)
        pickle.dump(pick_tup, open(f'../Pickles/{pick_name}.p', 'wb'))
                    
    return x_train, x_test, y_train, y_test, all_notes_dict